In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [32]:
spark.read.csv('test2.csv',header=True,inferSchema=True).show()
df_pyspark = spark.read.csv('test2.csv',header=True,inferSchema=False)

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [7]:
##Dropping columns
df_pyspark.drop('Name').show()

+----+----------+------+
| age|Experience|Salary|
+----+----------+------+
|  31|        10| 30000|
|  30|         8| 25000|
|  29|         4| 20000|
|  24|         3| 20000|
|  21|         1| 15000|
|  23|         2| 18000|
|null|      null| 40000|
|  34|        10| 38000|
|  36|      null|  null|
+----+----------+------+



In [8]:
#Dropping rows that contain null values
df_pyspark.na.drop().show() #Drops a row wherever a null value is present


+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [9]:
### how = any -->drop the row if there are any nulls. Any is the default
df_pyspark.na.drop(how='all').show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
|     null|  36|      null|  null|
+---------+----+----------+------+



In [10]:
##how=all-->only drop the row if all of the row values are null
df_pyspark.na.drop(how='any').show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [12]:
##Threshold --> At least a certain number of non-null values should be present. If there are fewer than the threshold-->drop
df_pyspark.na.drop(how='any',thresh=2).show()

+---------+----+----------+------+
|     Name| age|Experience|Salary|
+---------+----+----------+------+
|    Krish|  31|        10| 30000|
|Sudhanshu|  30|         8| 25000|
|    Sunny|  29|         4| 20000|
|     Paul|  24|         3| 20000|
|   Harsha|  21|         1| 15000|
|  Shubham|  23|         2| 18000|
|   Mahesh|null|      null| 40000|
|     null|  34|        10| 38000|
+---------+----+----------+------+



In [33]:
##Subset --> Dropping rows where there are na values in a specific column
df_pyspark.na.drop(how='any',subset=['Experience']).show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
|     null| 34|        10| 38000|
+---------+---+----------+------+



In [34]:
## Filling the missing values

#Fills it with the string 'Missing Values'
df_pyspark.na.fill('Missing Values').show()


+--------------+--------------+--------------+--------------+
|          Name|           age|    Experience|        Salary|
+--------------+--------------+--------------+--------------+
|         Krish|            31|            10|         30000|
|     Sudhanshu|            30|             8|         25000|
|         Sunny|            29|             4|         20000|
|          Paul|            24|             3|         20000|
|        Harsha|            21|             1|         15000|
|       Shubham|            23|             2|         18000|
|        Mahesh|Missing Values|Missing Values|         40000|
|Missing Values|            34|            10|         38000|
|Missing Values|            36|Missing Values|Missing Values|
+--------------+--------------+--------------+--------------+



In [35]:
## Filling the missing values

#Fills it with the string "Missing Values" only in certain columns
df_pyspark.na.fill('Missing Values',['Experience','age']).show()


+---------+--------------+--------------+------+
|     Name|           age|    Experience|Salary|
+---------+--------------+--------------+------+
|    Krish|            31|            10| 30000|
|Sudhanshu|            30|             8| 25000|
|    Sunny|            29|             4| 20000|
|     Paul|            24|             3| 20000|
|   Harsha|            21|             1| 15000|
|  Shubham|            23|             2| 18000|
|   Mahesh|Missing Values|Missing Values| 40000|
|     null|            34|            10| 38000|
|     null|            36|Missing Values|  null|
+---------+--------------+--------------+------+



In [38]:
#Filling the missing values with the mean or median of the present values

from pyspark.ml.feature import Imputer
df_pyspark = spark.read.csv('test2.csv',header=True,inferSchema=True)

imputer = Imputer(
    inputCols=['age', 'Experience', 'Salary'], 
    outputCols=["{}_imputed".format(c) for c in ['age', 'Experience', 'Salary']]
    ).setStrategy("median")

In [39]:
# Add imputation cols to df
imputer.fit(df_pyspark).transform(df_pyspark).show()

+---------+----+----------+------+-----------+------------------+--------------+
|     Name| age|Experience|Salary|age_imputed|Experience_imputed|Salary_imputed|
+---------+----+----------+------+-----------+------------------+--------------+
|    Krish|  31|        10| 30000|         31|                10|         30000|
|Sudhanshu|  30|         8| 25000|         30|                 8|         25000|
|    Sunny|  29|         4| 20000|         29|                 4|         20000|
|     Paul|  24|         3| 20000|         24|                 3|         20000|
|   Harsha|  21|         1| 15000|         21|                 1|         15000|
|  Shubham|  23|         2| 18000|         23|                 2|         18000|
|   Mahesh|null|      null| 40000|         29|                 4|         40000|
|     null|  34|        10| 38000|         34|                10|         38000|
|     null|  36|      null|  null|         36|                 4|         20000|
+---------+----+----------+-